In [1]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
from pathlib import Path
from gwp_uncertainties import add_bw_method_with_gwp_uncertainties

# Local files
from consumption_model_ch.import_databases import (
    import_exiobase_3,
    import_consumption_db,
)
from consumption_model_ch.consumption_fus import (
    add_consumption_activities,
    add_consumption_categories,
    add_consumption_sectors,
    add_archetypes_consumption,
    get_archetypes_scores_per_month,
    get_archetypes_scores_per_sector,
)
from consumption_model_ch.plot_archetypes import plot_archetypes_scores_yearly, plot_archetypes_scores_per_sector


In [2]:
# bd.projects.set_current("GSA for archetypes with exiobase")
bd.projects.set_current("GSA for archetypes")
# for key in bd.databases.keys():
#     del bd.databases['key']

In [4]:
# try:
#     del bd.databases["swiss consumption 1.0"]
# except:
#     pass
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent 3.8 cutoff

In [5]:
def import_all_databases(use_exiobase, add_activities=True, year='091011'):

    path_base = Path('/Users/akim/Documents/LCA_files/')

    directory_habe = path_base / 'HABE_2017/'
    fp_ecoinvent_38 = path_base / "ecoinvent_38_cutoff" / "datasets"
    # fp_ecoinvent_38 = "/Users/cmutel/Documents/lca/Ecoinvent/3.8/cutoff/datasets"
    fp_ecoinvent_33 = path_base / "ecoinvent_33_cutoff"/ "datasets"
    fp_exiobase = path_base / "exiobase_381_monetary" / "IOT_2015_pxp"
    # fp_archetypes = path_base / "heia" / "hh_archetypes_weighted_ipcc_091011.csv"

    ei38_name = "ecoinvent 3.8 cutoff"
    ex38_name = "exiobase 3.8.1 monetary"
    co_name = "swiss consumption 1.0"

    if use_exiobase:
        project = "GSA for archetypes with exiobase"
    else:
        project = "GSA for archetypes"
    bd.projects.set_current(project)

    # Import biosphere and ecoinvent databases
    if ei38_name not in bd.databases:
        bi.bw2setup()
        ei = bi.SingleOutputEcospold2Importer(fp_ecoinvent_38, ei38_name)
        ei.apply_strategies()
        assert ei.all_linked
        ei.write_database()

    exclude_databases = [
        'heia',
        'Agribalyse 1.2',
        'Agribalyse 1.3 - {}'.format(ei38_name),
    ]

    # Import exiobase
    if use_exiobase:
        import_exiobase_3(fp_exiobase, ex38_name)
    else:
        exclude_databases.append('exiobase 2.2')

    # Import consumption database
    import_consumption_db(
        directory_habe, co_name, fp_ecoinvent_33, exclude_databases, fp_exiobase,
    )

    # Add uncertainties to GWP values
    method = ("IPCC 2013", "climate change", "GWP 100a", "uncertain")
    if method not in bd.methods:
        add_bw_method_with_gwp_uncertainties()

    # Add functional units
    co = bd.Database(co_name)
    option = 'aggregated'
    if add_activities:
        add_consumption_activities(co_name, year, option=option,)
        add_consumption_categories(co_name)
        add_consumption_sectors(co_name, year)
        add_archetypes_consumption(co_name, year)

    # LCIA for average consumption
    co_average_act_name = f'ch hh average consumption {option}, years {year}'
    hh_average = [act for act in co if co_average_act_name == act['name']]
    assert len(hh_average) == 1
    demand_act = hh_average[0]
    lca = bc.LCA({demand_act: 1}, method)
    lca.lci()
    lca.lcia()
    print("{:8.3f}  {}".format(lca.score, demand_act['name']))

    # LCIA for all Swiss consumption sectors
    sectors = sorted([act for act in co if "sector" in act['name'].lower() and str(year) in act['name']])
    for demand_act in sectors:
        lca = bc.LCA({demand_act: 1}, method)
        lca.lci()
        lca.lcia()
        print("{:8.3f}  {}".format(lca.score, demand_act['name']))
        

def plot_archetypes(use_exiobase, year):
    # Add archetypes and compute total yearly scores per archetype
    if use_exiobase:
        project = "GSA for archetypes with exiobase"
    else:
        project = "GSA for archetypes"
    bd.projects.set_current(project)
    co_name = "swiss consumption 1.0"
    write_dir = Path("write_files") / project.lower().replace(" ", "_") / "archetype_scores"
    write_dir.mkdir(parents=True, exist_ok=True)

    method = ("IPCC 2013", "climate change", "GWP 100a", "uncertain")
    fp_archetypes_scores = write_dir / f"monthly_scores_{year}.pickle"
    archetypes_scores_monthly = get_archetypes_scores_per_month(co_name, year, method, fp_archetypes_scores)

    # Compare with Andi's results (reproduce Fig. 3 in Andi's data mining paper, top part)
    num_months_in_year = 12
    archetypes_scores_yearly = {
        archetype: score*num_months_in_year for archetype, score in archetypes_scores_monthly.items()
    }
    fig = plot_archetypes_scores_yearly(archetypes_scores_yearly)
#     fig.write_html(write_dir / "yearly_scores.html")
    fig.write_image(write_dir / f"yearly_scores_{year}.pdf")
    fig.show()

    # Compare with Andi's contributions per sectors (reproduce Fig. 3 in Andi's data mining paper, bottom part)
    fp_archetypes_scores_sectors = write_dir / f"monthly_scores_{year}.pickle"
    archetypes_scores_per_sector = get_archetypes_scores_per_sector(co_name, year, method, write_dir)
    fig = plot_archetypes_scores_per_sector(archetypes_scores_per_sector)
#     fig.write_html(write_dir / "sector_scores.html")
    fig.write_image(write_dir / "sector_scores.pdf")
    fig.show()


if __name__ == "__main__":

    print("Impacts WITHOUT exiobase")
    print("------------------------")
    use_ex = False
    year = '091011'
    import_all_databases(use_ex, False, year)
#     plot_archetypes(use_ex, year)

#     print("\n")
#     print("Impacts WITH exiobase")
#     print("---------------------")
#     use_ex = True
#     import_all_databases(use_ex, True)
#     plot_archetypes(use_ex)


Impacts WITHOUT exiobase
------------------------
Biosphere database already present!!! No setup is needed
ecoinvent 3.3 cutoff database already present!!! No import is needed
--> Consumption_db.xlsx already exists, reading it
Created consumption_db.xlsx file in BW format in 0.20 seconds
Extracted 1 worksheets in 0.23 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applied 0 strategies in 0.00 seconds
207 datasets
903 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 03/06/2022 17:22:29
  Finished: 03/06/2022 17:22:29
  Total time elapsed: 00:00:00
  CPU %: 106.00
  Memory %: 1.82
Created database: swiss consumption 1.0


AssertionError: 

In [6]:
bd.databases

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent 3.8 cutoff
	swiss consumption 1.0

In [27]:
co_name = "swiss consumption 1.0"
year = "091011"
option = 'aggregated'
co = bd.Database(co_name)
method = ("IPCC 2013", "climate change", "GWP 100a", "uncertain")
# add_consumption_activities(co_name, year, option=option,)
# add_consumption_categories(co_name)
# add_consumption_sectors(co_name, year)
add_archetypes_consumption(co_name, year)

Creating archetypes functional units


In [28]:
list(co)

['Tram, tickets and travelcards' (person kilometer, CH, None),
 'archetype_Q_consumption_years_091011' (1 month of consumption, CH, None),
 'Other purchased transport services' (CHF, CH, None),
 'archetype_F_consumption_years_091011' (1 month of consumption, CH, None),
 'Footwear for men' (CHF, CH, None),
 'Mobile telephony' (CHF, CH, None),
 'Margarine' (kilogram, CH, None),
 'Clothing and footwear sector, years 091011' (1 month of consumption, CH, None),
 'Small electric household appliances' (CHF, CH, None),
 'Whole milk' (litre, CH, None),
 'Non-durable products of cotton and paper for personal care' (CHF, CH, None),
 'Tinned or processed vegetables and mushrooms' (kilogram, CH, None),
 'Garments for women' (CHF, CH, None),
 'Onions' (kilogram, CH, None),
 'Furniture and furnishings, carpets and other floor coverings incl. repairs' (CHF, CH, None),
 'Milk-based beverages and other similar milk-based products' (litre, CH, None),
 'Pork, fresh or frozen' (kilogram, CH, None),
 'Finan

In [17]:
conact = [act for act in co if 'ch hh average consumption aggregated, years 091011' == act['name']][0]
lca = bc.LCA({conact: 1}, method)
lca.lci()
lca.lcia()
lca.score

1108.8133423074084

In [26]:
sectors = [act for act in co if 'sector' in act['name']]
for sector in sectors:
    lca = bc.LCA({sector: 1}, method)
    lca.lci()
    lca.lcia()
    print(lca.score, sector)

0.0 'Restaurants and hotels sector, years 091011' (1 month of consumption, CH, None)
0.0 'Fees sector, years 091011' (1 month of consumption, CH, None)
0.0 'Health sector, years 091011' (1 month of consumption, CH, None)
0.0 'Education sector, years 091011' (1 month of consumption, CH, None)
0.0 'Communication sector, years 091011' (1 month of consumption, CH, None)
131.46268042115344 'Durable goods sector, years 091011' (1 month of consumption, CH, None)
4.078184448203904 'Miscellaneous goods and services sector, years 091011' (1 month of consumption, CH, None)
351.92796975335386 'Housing, water, electricity, gas and other fuels sector, years 091011' (1 month of consumption, CH, None)
1.3041240213539935 'Recreation and culture sector, years 091011' (1 month of consumption, CH, None)
0.0 'Premiums for life insurance sector, years 091011' (1 month of consumption, CH, None)
0.0 'Clothing and footwear sector, years 091011' (1 month of consumption, CH, None)
0.0 'Other insurance premiums s

In [29]:
archetypes = [act for act in co if 'archetype' in act['name']]
# for sector in sectors:
#     lca = bc.LCA({sector: 1}, method)
#     lca.lci()
#     lca.lcia()
#     print(lca.score, sector)
archetypes

['archetype_H_consumption_years_091011' (1 month of consumption, CH, None),
 'archetype_N_consumption_years_091011' (1 month of consumption, CH, None),
 'archetype_B_consumption_years_091011' (1 month of consumption, CH, None),
 'archetype_A_consumption_years_091011' (1 month of consumption, CH, None),
 'archetype_OB_consumption_years_091011' (1 month of consumption, CH, None),
 'archetype_T_consumption_years_091011' (1 month of consumption, CH, None),
 'archetype_R_consumption_years_091011' (1 month of consumption, CH, None),
 'archetype_C_consumption_years_091011' (1 month of consumption, CH, None),
 'archetype_S_consumption_years_091011' (1 month of consumption, CH, None),
 'archetype_V_consumption_years_091011' (1 month of consumption, CH, None),
 'archetype_P_consumption_years_091011' (1 month of consumption, CH, None),
 'archetype_L_consumption_years_091011' (1 month of consumption, CH, None),
 'archetype_I_consumption_years_091011' (1 month of consumption, CH, None),
 'archetype

In [ ]:
%%time 
# Backup GSA project
bi.backup_project_directory("GSA for archetypes")
# bi.backup_project_directory("GSA for archetypes with exiobase")
# # Restore GSA project
# fp_gsa_project = path_base / "brightway2-project-GSA-backup.16-November-2021-11-50AM.tar.gz"
# if project not in bd.projects:
#     bi.restore_project_directory(fp_gsa_project)